In [ ]:
import numpy as np
import importlib

import pathlib
import sys

try:
    import chsimpy
except ImportError:
    _parentdir = pathlib.Path("./").resolve().parent
    sys.path.insert(0, str(_parentdir))
    import chsimpy
    #sys.path.remove(str(_parentdir))

In [ ]:
# if code has changed, reload here
importlib.reload(chsimpy)

importlib.reload(chsimpy.plotview)
importlib.reload(chsimpy.solution)
importlib.reload(chsimpy.model)
importlib.reload(chsimpy.controller)
importlib.reload(chsimpy.parameters)
importlib.reload(chsimpy.utils)
importlib.reload(chsimpy.mport)

In [ ]:
params = chsimpy.parameters.Parameters()
params.ntmax = 1500
params.render_target='gui'
controller = chsimpy.controller.Controller(params)
solution = controller.run()
controller.render()

In [ ]:
#SAlist = [controller.model.SA, controller.model.SA2, controller.model.SA3]
#(np.min(np.min(SAlist)), np.max(np.max(SAlist)))
#SAlist[2]

#controller.model.hat_U

# controller.model.params.Leig

# chsimpy.mport.gradient(controller.model.U, controller.params.delx)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
# matplotlib.artist.getp(fig.patch) # https://matplotlib.org/stable/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py

In [ ]:
U = solution.U
#Uhat = controller.model.hat_U
Ureal = np.real(U)
N = controller.model.params.N

# colormap, https://matplotlib.org/2.0.2/users/colormaps.html
cmap = colors.ListedColormap(['orange', 'yellow'])
boundaries = [0.0, solution.params.threshold, 1]
norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)

fig = plt.figure(figsize=(12,5))
# nrow, ncol, index
ax1 = fig.add_subplot(1, 2, 1)

#ax1.imshow(Ureal, cmap=cmap, norm=norm, aspect="equal")
ax1.imshow(Ureal, cmap="plasma", aspect="equal")

ax2 = fig.add_subplot(1, 2, 2) 

#ax.cla()
#ax2.set_xlim(0, N)
#ax2.set_ylim(0.5, 1)

# https://matplotlib.org/2.0.2/users/colormaps.html
ax2.plot(np.arange(0,N), Ureal[int(N / 2)+1,:])

display(fig)
#plt.pause(0.5)
matplotlib.pyplot.close()

In [ ]:
# https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389